<a href="https://colab.research.google.com/github/aaronblanco/practica-acuity/blob/main/PracticaAcuity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica Acuity

## Celda 1: Importación de Librerías

In [5]:

import warnings
from google.colab import drive
warnings.filterwarnings("ignore", category=DeprecationWarning)
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado
from sklearn.model_selection import KFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)

Mounted at /content/drive


## Celda 2: Carga de Datos

In [6]:
# ==========================================
# CARGA DE DATASETS
# ==========================================
# Definimos la ruta donde se encuentran los archivos
from pathlib import Path
DATA_DIR = Path('/content/drive/MyDrive/datos/')

# Cargamos cada dataset
try:
    df_submission = pd.read_csv(DATA_DIR / 'submission.csv')
    print("submission.csv cargado")

    df_test_data = pd.read_csv(DATA_DIR / 'test_kaggle.csv')
    print("datos de test cargado")

    df_train_data = pd.read_csv(DATA_DIR / 'train_kaggle.csv')
    print("datos de entrenamiento cargado")

except FileNotFoundError as e:
    print(f"Error: Archivo no encontrado - {e}")
    print("Verifica que todos los archivos estén en la carpeta correcta.")
except Exception as e:
    print(f"Error inesperado al cargar archivos: {e}")




submission.csv cargado
datos de test cargado
datos de entrenamiento cargado


## Celda 3: Limpieza Inicial y Definición del Objetivo

In [ ]:
# A. Eliminar filas donde no tengamos la respuesta (target 'acuity' nulo)
df = df.dropna(subset=['acuity'])

# B. Definir Target (y) y Features (X)
y = df['acuity']

# C. Seleccionar columnas útiles
# Eliminamos IDs (no aportan info médica), Fechas (complejas para este ejercicio) y Texto libre
cols_to_drop = ['stay_id', 'subject_id', 'hadm_id', 'subject_id_triage',
                'intime', 'outtime', 'chiefcomplaint', 'acuity']

X = df.drop(columns=cols_to_drop, errors='ignore')

print("Variables listas para procesar:", list(X.columns))